# Struggling with the computation complexity

In [1]:
import numpy as np
import pandas as pd
import ripser
from persim.persistent_entropy import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neighbors import NearestNeighbors

In [12]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

housing = fetch_california_housing()

X_all = pd.DataFrame(housing.data, columns = housing.feature_names)
y_all = housing['target']

X, _, y, _ = train_test_split(X_all, y_all, train_size=5000, random_state=42)

X = X.reset_index(drop=True)
y = pd.Series(y).reset_index(drop=True)

In [31]:
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index= X.index, columns= X.columns)
X_final = X.iloc[:, :8].to_numpy()

X.iloc[:, :8].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,-1.165910,-1.070219,-0.262989,0.076520,0.188266,-0.388556,3.032089,-2.403671
1,-0.897428,-1.630627,-0.274419,-0.147923,-0.197086,0.760507,0.137286,0.124272
2,0.319184,-1.150277,0.576531,-0.167376,0.656260,-0.025571,1.430298,-1.674849
3,0.532011,-1.870802,0.151656,-0.093335,-0.182229,0.175792,1.078097,-0.808417
4,-1.337464,-0.830044,-0.307826,-0.017700,-0.012302,2.003783,-1.411434,1.230247


In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.5, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

RMSE: 0.6863037922540264
R-square: 0.6296337782599915


# Add Entropy featureX

In [24]:
entropy_feature = []
for i in range(X.shape[0]):
    if (i + 1) % 50 == 0:
        print(f"Processing row {i + 1} out of {X.shape[0]}")
        
    data_remove = X.drop(index=i)
    dgm = ripser.ripser(data_remove, maxdim=0)['dgms'][0]
    p_entropy = persistent_entropy(dgm)
    entropy_feature.append(p_entropy)

Processing row 50 out of 1000
Processing row 100 out of 1000
Processing row 150 out of 1000
Processing row 200 out of 1000
Processing row 250 out of 1000
Processing row 300 out of 1000
Processing row 350 out of 1000
Processing row 400 out of 1000
Processing row 450 out of 1000
Processing row 500 out of 1000
Processing row 550 out of 1000
Processing row 600 out of 1000
Processing row 650 out of 1000
Processing row 700 out of 1000
Processing row 750 out of 1000
Processing row 800 out of 1000
Processing row 850 out of 1000
Processing row 900 out of 1000
Processing row 950 out of 1000
Processing row 1000 out of 1000


In [29]:
X['topo_feature'] = [x.tolist()[0] if isinstance(x, np.ndarray) else x for x in entropy_feature]
X['topo_feature'] = scaler.fit_transform(X[['topo_feature']])
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,topo_feature
0,-1.165910,-1.070219,-0.262989,0.076520,0.188266,-0.388556,3.032089,-2.403671,-0.428133
1,-0.897428,-1.630627,-0.274419,-0.147923,-0.197086,0.760507,0.137286,0.124272,-0.155814
2,0.319184,-1.150277,0.576531,-0.167376,0.656260,-0.025571,1.430298,-1.674849,-0.098502
3,0.532011,-1.870802,0.151656,-0.093335,-0.182229,0.175792,1.078097,-0.808417,-0.044423
4,-1.337464,-0.830044,-0.307826,-0.017700,-0.012302,2.003783,-1.411434,1.230247,0.004009


In [30]:
X_final = X.iloc[:, :9].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.5, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

RMSE: 0.8125377810376255
R-square: 0.48085855527198473
